In [1]:
!pip install numpy
!pip install matplotlib

# 📜 Açıklama

## 1. Fiziksel Model

Genel Görelilik'in zayıf alan limitinde, ışık ışınının Güneş'in kütleçekim alanında bükülme açısı:

$$\alpha \approx \frac{4GM}{c^2 b}$$

**Parametreler:**
* **G** → Evrensel kütle çekim sabiti
* **M** → Güneş kütlesi
* **c** → Işık hızı
* **b** → En yakın yaklaşma mesafesi

Biz burada **tam Einstein denklemlerini çözmeyeceğiz** çünkü görselleştirme için yeterli olan basitleştirilmiş bir eğri (yani parametrelenmiş bir yol) kullanacağız.

## 2. Simülasyon Mantığı

* Uzay-zaman ızgarasını oluşturuyoruz (x, y eksenlerinde eğrilik olacak)
* Işık ışınını başlangıçta düz çizgi olarak tanımlayıp, Güneş'e yaklaşırken yukarıdaki formüle göre yörüngesini büküyoruz
* Matplotlib `FuncAnimation` ile hem ızgarayı hem de ışık yolunu zamanla hareket ettiriyoruz


In [ ]:
# === 1. Kütüphaneler ===
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# === 2. Fiziksel Sabitler ===
G = 6.67430e-11
M_sun = 1.989e30
c = 3e8
R_sun = 6.96e8  # metre

# === 3. Uzay-zaman ızgarası ===
grid_size = 50
space = np.linspace(-3, 3, grid_size)
X, Y = np.meshgrid(space, space)
R = np.sqrt(X**2 + Y**2)
Z = -1.0 / (R + 0.2)  # bükülme etkisi

# === 4. Işık ışını ===
def light_path(b=1.2):
    """ Işık ışınının görelilik etkisiyle bükülmüş yolunu döndürür """
    x = np.linspace(-3, 3, 1000)
    y = np.zeros_like(x) + b
    
    # Deflection angle (rad)
    alpha = (4 * G * M_sun) / (c**2 * b * R_sun)
    
    # Işık güneşe yaklaştığında kavis oluştur
    curve = alpha * np.exp(-x**2 / 0.3)
    y_curve = y + curve
    
    return x, y_curve

x_light, y_light = light_path()
z_light = np.zeros_like(x_light)

# === 5. Güneş küresi ===
u = np.linspace(0, 2 * np.pi, 40)
v = np.linspace(0, np.pi, 40)
xs = 0.5 * np.outer(np.cos(u), np.sin(v))
ys = 0.5 * np.outer(np.sin(u), np.sin(v))
zs = 0.5 * np.outer(np.ones_like(u), np.cos(v))

# === 6. Animasyon ayarları ===
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

def update(frame):
    ax.cla()
    
    # Uzay-zaman yüzeyi
    ax.plot_surface(X, Y, Z, cmap="plasma", alpha=0.7, edgecolor='none')
    
    # Güneş
    ax.plot_surface(xs, ys, zs, color="yellow", alpha=1.0)
    
    # Işık ışını
    idx = min(frame*8, len(x_light))
    ax.plot(x_light[:idx], y_light[:idx], z_light[:idx], color="white", lw=2)
    
    # Ayarlar
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_zlim(-1.5, 1.5)
    ax.view_init(elev=30, azim=frame*2)  # Kamera döndürme
    ax.set_axis_off()
    
    return []

anim = FuncAnimation(fig, update, frames=100, interval=60, blit=False)

from IPython.display import HTML
HTML(anim.to_jshtml())
